## Customizing models

Transformer 模型的被设计为具有可定制性。每个模型的代码都完整地包含在 Transformers 代码库的模型子文件夹中。每个文件夹都包含一个 modeling.py 文件和一个 configuration.py 文件。复制这些文件即可开始对模型进行定制。

本指南将向您展示如何自定义 ResNet 模型、启用 AutoClass 支持，并将其分享到 Hub 上。

## Configuration

由基础的 PretrainedConfig 类给出的配置包含了构建模型所需的所有必要信息。在这里，您可以配置自定义 ResNet 模型的属性。不同的属性会生成不同类型的 ResNet 模型。

自定义配置的主要规则为：

1. 自定义配置必须继承自 `PretrainedConfig` 类。这能确保自定义模型具备 `Transformers` 模型的所有功能，例如 `from_pretrained()`、`save_pretrained()` 和 `push_to_hub()` 等。
2. `PretrainedConfig` 的 `__init__` 方法必须接受任何`kwargs`，并将它们传递给父类的 `__init__` 方法。`PretrainedConfig` 拥有的字段比您自定义配置中设置的字段多，因此当您使用 `from_pretrained()` 加载配置时，您的配置需要接受这些字段，并将它们传递给父类。

检查某些参数的有效性是有用的。在下面的示例中，实现了一个检查以确保 `block_type` 和 `stem_type` 属于预定义的值之一。

将 `model_type` 添加到配置类中以启用 `AutoClass` 支持。

In [ ]:
from transformers import PretrainedConfig
from typing import List

class ResnetConfig(PretrainedConfig):
    # 将model_type添加到配置类中以启用AutoClass支持。
    model_type = "resnet"

    def __init__(
        self,
        block_type="bottleneck",
        layers: List[int] = [3, 4, 6, 3],
        num_classes: int = 1000,
        input_channels: int = 3,
        cardinality: int = 1,
        base_width: int = 64,
        stem_width: int = 64,
        stem_type: str = "",
        avg_down: bool = False,
        **kwargs,
    ):
        if block_type not in ["basic", "bottleneck"]:
            raise ValueError(f"`block_type` must be 'basic' or bottleneck', got {block_type}.")
        if stem_type not in ["", "deep", "deep-tiered"]:
            raise ValueError(f"`stem_type` must be '', 'deep' or 'deep-tiered', got {stem_type}.")

        self.block_type = block_type
        self.layers = layers
        self.num_classes = num_classes
        self.input_channels = input_channels
        self.cardinality = cardinality
        self.base_width = base_width
        self.stem_width = stem_width
        self.stem_type = stem_type
        self.avg_down = avg_down
        super().__init__(**kwargs)

RuntimeError: Trying to override a python impl for DispatchKey.Meta on operator aten::broadcast_tensors

使用 `save_pretrained()` 函数将配置保存到您自定义模型文件夹 `custom-resnet` 中的一个 JSON 文件里。

In [ ]:
resnet50d_config = ResnetConfig(block_type="bottleneck", stem_width=32, stem_type="deep", avg_down=True)
resnet50d_config.save_pretrained("custom-resnet")

## Model

使用自定义的 ResNet 配置，您现在可以创建和自定义模型。该模型继承了基础的 PreTrainedModel 类。与 PretrainedConfig 类似，从 PreTrainedModel 继承并在初始化超类时传入配置，可将 Transformers 的功能（如保存和加载）扩展到自定义模型。

Transformer 模型遵循在 __init__ 方法中接受配置对象的惯例。这会将整个配置传递给模型的子层，而不是将配置对象拆分为多个参数分别传递给子层。

以这种方式编写模型能够生成更简洁的代码，并且任何超参数都有明确的来源。这也有助于更轻松地复用来自其他 Transformer 模型的代码。

您将创建两个 ResNet 模型，一个是输出隐藏状态的精简版 ResNet 模型，另一个是带有图像分类头的 ResNet 模型。

### ResnetModel

定义块类型与类之间的映射关系。其他所有内容都是通过将配置类传递给 ResNet 模型类来创建的。

In [1]:
from transformers import PreTrainedModel
from timm.models.resnet import BasicBlock, Bottleneck, ResNet
from .configuration_resnet import ResnetConfig

BLOCK_MAPPING = {"basic": BasicBlock, "bottleneck": Bottleneck}

class ResnetModel(PreTrainedModel):
    config_class = ResnetConfig

    def __init__(self, config):
        super().__init__(config)
        block_layer = BLOCK_MAPPING[config.block_type]
        self.model = ResNet(
            block_layer,
            config.layers,
            num_classes=config.num_classes,
            in_chans=config.input_channels,
            cardinality=config.cardinality,
            base_width=config.base_width,
            stem_width=config.stem_width,
            stem_type=config.stem_type,
            avg_down=config.avg_down,
        )

    def forward(self, tensor):
        return self.model.forward_features(tensor)

KeyboardInterrupt: 